Kaggle URL: [YoloWorld Train Distillation](https://www.kaggle.com/code/windstorm1412/yoloworld-train-distillation)

In [ ]:
!pip -q install ultralytics==8.3.27 open_clip_torch==2.24.0
!pip uninstall -y ray ray[default] ray[tune] >/dev/null 2>&1 || true
!pip install -q "numpy<2.0"
!pip install -q protobuf==3.20.3 tensorboard==2.14.0
!pip install -q filterpy

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import os, json
import requests
import math
import torch
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import cv2
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import yaml
import re
import gc
from ultralytics import YOLOWorld

# --- Cấu hình ---
WORK_DIR = "/kaggle/input/yolo-world-distill"
DISTILL_DIR = os.path.join(WORK_DIR, "yolo_world_distill")
DATA_YAML = os.path.join(WORK_DIR, "data.yaml")
IMG_PATH = os.path.join(DISTILL_DIR, "images")
LBL_PATH = os.path.join(DISTILL_DIR, "labels")

In [ ]:
!cp $DATA_YAML /kaggle/working/
DATA_YAML = "/kaggle/working/data.yaml"
print(DATA_YAML)

In [ ]:
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

# 1. Cập nhật giá trị trong từ điển bằng các biến đường dẫn bạn đã khai báo
# Sử dụng IMG_PATH vì YOLO cần đường dẫn đến thư mục chứa ảnh
data_config['train'] = IMG_PATH
data_config['val'] = IMG_PATH

# 2. Ghi đè lại nội dung đã sửa vào file data.yaml
with open(DATA_YAML, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

# 3. Kiểm tra lại kết quả
print("✅ Đã cập nhật đường dẫn thành công!")
print(f"Đường dẫn Train mới: {data_config['train']}")
print(f"Đường dẫn Val mới: {data_config['val']}")
# Lấy lại danh sách class từ key 'names'
class_list = data_config.get('names', [])

print(f"✅ Đã khôi phục class_list: {class_list}")
print(f"📊 Số lượng class: {len(class_list)}")

In [ ]:
# Load mô hình YOLO-World bản nhỏ (S) hoặc trung bình (M) để làm nhẹ hệ thống
model = YOLOWorld('yolov8s-world.pt') 

# Thiết lập danh sách từ khóa mà mô hình cần tập trung (từ BLIP caption)
model.set_classes(class_list)

# Huấn luyện mô hình với dữ liệu từ Teacher
model.train(
    data="/kaggle/working/data.yaml",
    epochs=50,       # Giới hạn tối đa là 50
    patience=5,      # Nếu sau 5 epoch liên tiếp mà mAP không tăng, sẽ tự dừng
    imgsz=640,
    batch=16,
    device=0,
    save=True,       # Lưu lại mô hình tốt nhất
    project="yolo_world_drone",
)